##### API"Links":"https://aviationstack.com/documentation
##### "Build"Data"lake"from"scratch":"https://towardsdatascience.com/how-to-build-a-data-lake-from-scratch-part-1-the-setup-34ea1665a06e

In [1]:
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import logging
from pyspark.sql.types import *

In [2]:
load_dotenv("../properties/.env")

True

In [3]:
logging.basicConfig(format="%(name)s - %(levelname)s - %(asctime)s - %(message)s", level=logging.INFO)

In [4]:
spark = SparkSession.builder.master("local[*]").appName("flights-app").config("spark.sql.warehouse.dir", "../data/spark-warehouse").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/11/27 16:28:55 WARN Utils: Your hostname, MSIpowaaa resolves to a loopback address: 127.0.1.1; using 172.25.165.28 instead (on interface eth0)
23/11/27 16:28:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/27 16:28:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.parquet(f"../data/silver/2023-11-26_data.parquet")

In [6]:
df.show()

23/11/27 16:29:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-------------+--------------------+------------------+--------+--------+------------+--------+---------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+--------+--------+------------+--------+---------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+------------+------------+-------------+-----------+-----------+-----------------------+-----------------------+-----------------------+------------------------+----------------------+----------------------+
|        flight_date|flight_status|         dep_airport|      dep_timezone|dep_iata|dep_icao|dep_terminal|dep_gate|dep_delay|      dep_scheduled|      dep_estimated|         dep_actual|dep_estimated_runway|  dep_actual_runway|      arr_timezone|arr_iata|arr_icao|arr_terminal|arr_gate|arr_delay|      arr_scheduled|      arr_estimated|         arr_actual|arr_estimated_runway

In [7]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show() 

+-----------+-------------+-----------+------------+--------+--------+------------+--------+---------+-------------+-------------+----------+--------------------+-----------------+------------+--------+--------+------------+--------+---------+-------------+-------------+----------+--------------------+-----------------+------------+------------+------------+-------------+-----------+-----------+-----------------------+-----------------------+-----------------------+------------------------+----------------------+----------------------+
|flight_date|flight_status|dep_airport|dep_timezone|dep_iata|dep_icao|dep_terminal|dep_gate|dep_delay|dep_scheduled|dep_estimated|dep_actual|dep_estimated_runway|dep_actual_runway|arr_timezone|arr_iata|arr_icao|arr_terminal|arr_gate|arr_delay|arr_scheduled|arr_estimated|arr_actual|arr_estimated_runway|arr_actual_runway|airline_name|airline_iata|airline_icao|flight_number|flight_iata|flight_icao|codeshared_airline_name|codeshared_airline_iata|codeshared_a

In [8]:
df.count()

100

In [9]:
df.select(min("dep_delay")).show()

+--------------+
|min(dep_delay)|
+--------------+
|             0|
+--------------+



In [10]:
df.select("dep_terminal").distinct().show()

+------------+
|dep_terminal|
+------------+
|           3|
|           D|
|           A|
|          T2|
|      Uknown|
|           1|
|           I|
|           4|
|           2|
+------------+



In [11]:
df.schema["dep_estimated_runway"].dataType

StringType()

In [12]:
df.printSchema()

root
 |-- flight_date: timestamp (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- dep_airport: string (nullable = true)
 |-- dep_timezone: string (nullable = true)
 |-- dep_iata: string (nullable = true)
 |-- dep_icao: string (nullable = true)
 |-- dep_terminal: string (nullable = true)
 |-- dep_gate: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- dep_scheduled: timestamp (nullable = true)
 |-- dep_estimated: timestamp (nullable = true)
 |-- dep_actual: string (nullable = true)
 |-- dep_estimated_runway: string (nullable = true)
 |-- dep_actual_runway: string (nullable = true)
 |-- arr_timezone: string (nullable = true)
 |-- arr_iata: string (nullable = true)
 |-- arr_icao: string (nullable = true)
 |-- arr_terminal: string (nullable = true)
 |-- arr_gate: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- arr_scheduled: timestamp (nullable = true)
 |-- arr_estimated: timestamp (nullable = true)
 |-- arr_actual: string (n